In [1]:
%logstop
%logstart -rtq ~/.logs/DS_IO.py append
import seaborn as sns
sns.set()

In [3]:
import expectexception

In [4]:
# Delete files generated from running the notebook
!rm -f data/my_data.txt
!rm -f data/fail.txt

# Importing and Exporting Data

<!-- requirement: data/sample.txt -->
<!-- requirement: data/csv_sample.txt -->
<!-- requirement: data/bad_csv.csv -->

So far we've only dealt with data that we have created within Python. Generating random data is helpful for testing out ideas, but we want to work with real data. Most often that data will be stored in a file, either locally on the computer or online. In this notebook we'll learn how to read and write data to files.

## Python file handles (`open`)

In Python we interact with files on disk using the commands `open` and `close`. We've included a file in the `data` folder called `sample.txt`. Let's open it and read its contents

In [5]:
f = open('./data/sample.txt', 'r')
data = f.read()
f.close()

print(data)
print(f)

Hello!
Congratulations!
You've read in data from a file.
<_io.TextIOWrapper name='./data/sample.txt' mode='r' encoding='UTF-8'>


Notice that we `open` the file  and assign it to `f`, `read` the data from `f`, and then close `f`. What is `f`? It's called a **file handle**. It's an object that connects Python to the file we `open`. We `read` the data using this connection, and then once we're done with `close` the connection. It's a good habit to `close` a file handle once we're done with it, so usually we will do it automatically using Python's `with` keyword. 

In [6]:
# f is automatically closed
# at the end of the body of the with statement
with open('./data/sample.txt', 'r') as f:
    print(f.read())

print(f)

Hello!
Congratulations!
You've read in data from a file.
<_io.TextIOWrapper name='./data/sample.txt' mode='r' encoding='UTF-8'>


We can also read individual lines of a file.

In [7]:
with open('./data/sample.txt', 'r') as f:
    print(f.readline())
    print(f.readline())
    print(f.readline())

Hello!

Congratulations!

You've read in data from a file.


In [8]:
with open('./data/sample.txt', 'r') as f:
    print(f.readlines())
    print(type(f.readlines()))

['Hello!\n', 'Congratulations!\n', "You've read in data from a file."]
<class 'list'>


In [9]:
with open('./data/sample.txt', 'r') as f:
    print(f.read())
    print(type(f.read()))

Hello!
Congratulations!
You've read in data from a file.
<class 'str'>


Writing data to files is very similar. The main difference is when we `open` the file, we will use the `'w'` flag instead of `'r'`.

In [10]:
with open('./data/my_data.txt', 'w') as f:
    f.write('This is a new file.\n')
    f.write('I am practicing writing data to disk.')

with open('./data/my_data.txt', 'r') as f:
    my_data = f.read()

print(my_data)

This is a new file.
I am practicing writing data to disk.


No matter how often I execute the above cell, the same output gets printed. Opening the file with the `'w'` flag will overwrite the contents of the file. If we want to add to what is already in the file, we have to open the file with the `'a'` flag (`'a'` stands for _append_).

In [11]:
with open('./data/my_data.txt', 'a') as f:
    f.write('\nAdding a new line to the file.')

with open('./data/my_data.txt', 'r') as f:
    my_data = f.read()

print(my_data)

This is a new file.
I am practicing writing data to disk.
Adding a new line to the file.


We always need to be careful when writing to disk, because we could overwrite or alter data by accident. It is also easy to encounter errors when working with files, because we might not know ahead of time if the file we're trying to access exists, or we might mix up the `'r'`, `'w'`, and `'a'` flags.

In [12]:
%%expect_exception IOError

# if a file doesn't exist
# we can't open it for reading
# (but we can open it for writing)

with open('./data/fail.txt', 'r') as f:
    f.read()

---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
<ipython-input-12-3a04e58b575a> in <module>
      3 # (but we can open it for writing)
      4 
----> 5 with open('./data/fail.txt', 'r') as f:
      6     f.read()

FileNotFoundError: [Errno 2] No such file or directory: './data/fail.txt'


In [13]:
#%%expect_exception IOError

# we can't read a file open for writing

with open('./data/fail.txt', 'w') as f:
    f.write('Hi dumbass')

In [14]:
%%expect_exception IOError

# and we can't write to a file open for reading

with open('./data/sample.txt', 'r') as f:
    f.write('This will fail')

---------------------------------------------------------------------------
UnsupportedOperation                      Traceback (most recent call last)
<ipython-input-14-776a79381859> in <module>
      2 
      3 with open('./data/sample.txt', 'r') as f:
----> 4     f.write('This will fail')

UnsupportedOperation: not writable


Can we prevent some of these errors? How do we find out what files are on disk?

## `os` module

Python has a module for navigating the computer's file system called `os`. There are many useful tools in the `os` module, but there are two functions that are most useful for finding files.

In [15]:
import os

# list the contents of the current directory
# ('.' refers to the current directory)
os.listdir('.')

['DS_SQL.ipynb',
 'data',
 'miniprojects',
 'DS_Basic_DS_Modules.ipynb',
 'DS_IO.ipynb',
 '.ipynb_checkpoints',
 'DS_Data_Munging.ipynb',
 'DS_Classes_and_ORM.ipynb',
 '.done',
 'DS_Intro_Statistics.ipynb',
 'DS_Pandas.ipynb']

The command `listdir` is the simpler of the two functions we'll cover. It simply lists the contents of the directory path we specify. When we pass `'.'` as the argument, `listdir` will look in the current directory. It lists all the Jupyter notebooks we're using for the course, as well as the `data` subdirectory. We could find out what's in the `data` subdirectory by looking in `'./data'`.

In [16]:
os.listdir('./data')

['short_text.txt',
 'csv_sample.txt',
 'PEP_2016_PEPANNRES.csv',
 'fail.txt',
 'products.csv',
 'orders.csv',
 'my_data.txt',
 'bad_csv.csv',
 'customers.csv',
 'short_text.txt.gz',
 'eog_djvu_scrambled.txt.gz',
 'eog_djvu.txt.gz',
 'yelp.json.gz',
 'cia_factbook.csv',
 'pd_write.csv',
 'library.json',
 'eog_djvu.txt',
 'sample.txt']

What if we wanted to find all the files and subdirectories below a directory somewhere on our computer? With `listdir` we only see the files and subdirectories under the particular directory we're looking in. We cannot use `listdir` to automatically search through subdirectories. For this we need to use `walk`, which "walks" through all the subdirectories below our chosen directory. We won't cover `walk` in this course, but it's one of the very useful tools (along with the `os.path` sub-module) for working with files in Python, particularly if you are working with many different data files at once.

## CSV files

One of the simplest and most common formats for saving data is as comma-separated values (CSV).

In [17]:
with open('./data/csv_sample.txt', 'r') as f:
    csv = f.read()

print(csv)

index,name,age
0,Dylan,28
1,Terrence,54
2,Mya,31



In [18]:
import pandas as pd

csv_ = pd.read_csv('./data/csv_sample.txt', index_col=0)
print(csv_)

           name  age
index               
0         Dylan   28
1      Terrence   54
2           Mya   31


This format is often used to represent tables of data. Usually a CSV will have rows (separated by newline characters, `'\n'`) and columns (separated by commas). Otherwise they are no different from any other text file. We can use the special formatting of a CSV to create a list of lists representing the table.

In [19]:
list_table = []
with open('./data/csv_sample.txt', 'r') as f:
    for line in f.readlines():
        list_table.append(line.strip().split(','))

list_table

[['index', 'name', 'age'],
 ['0', 'Dylan', '28'],
 ['1', 'Terrence', '54'],
 ['2', 'Mya', '31']]

In [20]:
from collections import namedtuple
Row = namedtuple('row', ['index', 'name', 'age'])

def parse_line(line):
    row = line.strip().split(',')
    if row[0] == 'index':
        return row
    return Row(int(row[0]), row[1], int(row[2]))

list_table = []
with open('./data/csv_sample.txt', 'r') as f:
    for line in f:
        list_table.append(parse_line(line))

list_table

[['index', 'name', 'age'],
 row(index=0, name='Dylan', age=28),
 row(index=1, name='Terrence', age=54),
 row(index=2, name='Mya', age=31)]

However, we can work with tabular data much more easily in a Pandas DataFrame. Pandas provides a `read_csv` method to read the data directly into a DataFrame.

In [21]:
import pandas as pd

df = pd.read_csv('./data/csv_sample.txt', index_col=0)
df

,name,age
index,,
0,Dylan,28
1,Terrence,54
2,Mya,31


The `read_csv` method is very flexible to deal with the formatting of different data sets. Some data sets will include column headers while others may not. Some data sets will include an index while others may not. Some data sets may have values separated by tabs, semicolons, or other characters instead of commas. There are options in the `read_csv` method for dealing with all of these. You can read about them in the [Pandas documentation on `read_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html). We'll also discuss it further in the [Pandas notebook](DS_Pandas.ipynb).

In [22]:
# an example of downloading
# and importing real data using `read_csv`

if 'factbook.csv' not in os.listdir('./data/'):
    !wget -P ./data/ https://www.openintro.org/data/csv/cia_factbook.csv

countries = pd.read_csv('./data/cia_factbook.csv')
countries.head()

--2021-07-29 00:56:36--  https://www.openintro.org/data/csv/cia_factbook.csv
Resolving www.openintro.org (www.openintro.org)... 192.185.65.127
Connecting to www.openintro.org (www.openintro.org)|192.185.65.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16939 (17K) [text/csv]
Saving to: ‘./data/cia_factbook.csv.1’

cia_factbook.csv.1  100%[===================>]  16.54K  --.-KB/s    in 0.04s   

2021-07-29 00:56:36 (402 KB/s) - ‘./data/cia_factbook.csv.1’ saved [16939/16939]



,country,area,birth_rate,death_rate,infant_mortality_rate,internet_users,life_exp_at_birth,maternal_mortality_rate,net_migration_rate,population,population_growth_rate
0,Russia,17098242.0,11.87,13.83,7.08,40853000.0,70.16,34.0,1.69,1.424703e+08,-0.03
1,Canada,9984670.0,10.29,8.31,4.71,26960000.0,81.67,12.0,5.66,3.483484e+07,0.76
2,United States,9826675.0,13.42,8.15,6.17,245000000.0,79.56,21.0,2.45,3.188921e+08,0.77
3,China,9596960.0,12.17,7.44,14.79,389000000.0,75.15,37.0,-0.32,1.355693e+09,0.44
4,Brazil,8514877.0,14.72,6.54,19.21,75982000.0,73.28,56.0,-0.15,2.026568e+08,0.80


In [23]:
# we can also use pandas to write CSV
# using the DataFrame's to_csv method

pd.DataFrame({'a': [0, 3, 10], 'b': [True, True, False]}).to_csv('./data/pd_write.csv')

pd.read_csv('./data/pd_write.csv', index_col=0)

,a,b
0,0,True
1,3,True
2,10,False


Sometimes a CSV won't be perfect. For example, maybe different rows have different numbers of commas. This makes it difficult to interpret the contents of the file as a table.

In [24]:
# the 3rd line only has 2 "columns"

!cat ./data/bad_csv.csv

index,name,age
0,Dylan,27
1,54
2,Mya,31

In [25]:
# what happens if we try to read this
# into a DataFrame using read_csv?

pd.read_csv('./data/bad_csv.csv', index_col = 0)

,name,age
index,,
0,Dylan,27.0
1,54,NaN
2,Mya,31.0


Pandas' `read_csv` method will do its best to construct a table out of a poorly formatted CSV, but it may make mistakes. For example, 54 was interpreted as a name instead of as an age, because there were only 2 columns in that line of the file. Data sets will often contain mistakes like bad formatting, missing data, or typos.

**Question:** How could we fix the badly formatted CSV so it would work with `read_csv`?

## JSON

JSON stands for JavaScript Object Notation. JavaScript is a common language for creating web applications, and JSON files are used to collect and transmit information between JavaScript applications. As a result, a lot of data on the internet exists in the JSON file format. For example, Twitter and Google Maps use JSON.

A JSON file is essentially a data structure built out of nested dictionaries and lists. Let's make our own example and then we'll examine an example downloaded from the internet.

In [26]:
book1 = {'title': 'The Prophet',
         'author': 'Khalil Gibran',
         'genre': 'poetry',
         'tags': ['religion', 'spirituality', 'philosophy', 'Lebanon', 'Arabic', 'Middle East'],
         'book_id': '811.19',
         'copies': [{'edition_year': 1996,
                     'checkouts': 486,
                     'borrowed': False},
                    {'edition_year': 1996,
                     'checkouts': 443,
                     'borrowed': False}]
         }
         
book2 = {'title': 'The Little Prince',
         'author': 'Antoine de Saint-Exupery',
         'genre': 'children',
         'tags': ['fantasy', 'France', 'philosophy', 'illustrated', 'fable'],
         'id': '843.912',
         'copies': [{'edition_year': 1983,
                     'checkouts': 634,
                     'borrowed': True,
                     'due_date': '2017/02/02'},
                    {'edition_year': 2015,
                     'checkouts': 41,
                     'borrowed': False}]
         }

library = [book1, book2]
library

[{'title': 'The Prophet',
  'author': 'Khalil Gibran',
  'genre': 'poetry',
  'tags': ['religion',
   'spirituality',
   'philosophy',
   'Lebanon',
   'Arabic',
   'Middle East'],
  'book_id': '811.19',
  'copies': [{'edition_year': 1996, 'checkouts': 486, 'borrowed': False},
   {'edition_year': 1996, 'checkouts': 443, 'borrowed': False}]},
 {'title': 'The Little Prince',
  'author': 'Antoine de Saint-Exupery',
  'genre': 'children',
  'tags': ['fantasy', 'France', 'philosophy', 'illustrated', 'fable'],
  'id': '843.912',
  'copies': [{'edition_year': 1983,
    'checkouts': 634,
    'borrowed': True,
    'due_date': '2017/02/02'},
   {'edition_year': 2015, 'checkouts': 41, 'borrowed': False}]}]

We have two books in our `library`. Both books have some common properties: a title, an author, an id, and tags. Each book can have several tags, so we store that data as a list. Additionally, there can be multiple copies of each book, and each copy also has some unique information like the year it was printed and how many times it's been checked out. Notice that if a book is checked out, it also has a due date. It's convenient to store the information about the multiple copies as a list of dictionaries within the dictionary about the book, because every copy shares the same title, author, etc.

This structure is typical of JSON files. It has the advantage of reducing redundancy of data. We only store the author and title once, even though there are multiple copies of the book. Also, we don't store a due date for copies that aren't checked out.

If we were to put this data in a table, we would have to duplicate a lot of information. Also, since only one copy in our library is checked out, we also have a column with a lot of missing data.

|index|title|author|id|genre|tags|edition_year|checkouts|borrowed|due_date|
|:---:|:---:|:----:|::|:---:|:--:|:----------:|:-------:|:------:|:------:|
|0|The Prophet|Khalil Gibran|811.19|poetry|religion, spirituality, philosophy, Lebanon, Arabic, Middle East|1996|486|False|Null|
|1|The Prophet|Khalil Gibran|811.19|poetry|religion, spirituality, philosophy, Lebanon, Arabic, Middle East|1996|443|False|Null|
|2|The Little Prince|Antoine de Saint-Exupery|843.912|children|fantasy, France, philosophy, illustrated, fable|1983|634|True|2017/02/02|
|3|The Little Prince|Antoine de Saint-Exupery|843.912|children|fantasy, France, philosophy, illustrated, fable|2015|41|False|Null|

This is very wasteful. Since JSON files are meant to be shared quickly over the internet, it is important that they are small to reduce the amount of resources needed to store and transmit them.

We can write our `library` to disk using the `json` module.

In [27]:
import json

with open('./data/library.json', 'w') as f:
    json.dump(library, f, indent=2)

In [28]:
!cat ./data/library.json

[
  {
    "title": "The Prophet",
    "author": "Khalil Gibran",
    "genre": "poetry",
    "tags": [
      "religion",
      "spirituality",
      "philosophy",
      "Lebanon",
      "Arabic",
      "Middle East"
    ],
    "book_id": "811.19",
    "copies": [
      {
        "edition_year": 1996,
        "checkouts": 486,
        "borrowed": false
      },
      {
        "edition_year": 1996,
        "checkouts": 443,
        "borrowed": false
      }
    ]
  },
  {
    "title": "The Little Prince",
    "author": "Antoine de Saint-Exupery",
    "genre": "children",
    "tags": [
      "fantasy",
      "France",
      "philosophy",
      "illustrated",
      "fable"
    ],
    "id": "843.912",
    "copies": [
      {
        "edition_year": 1983,
        "checkouts": 634,
        "borrowed": true,
        "due_date": "2017/02/02"
      },
      {
        "edition_year": 2015,
        "checkouts": 41,
        "borrowed": false
      }
    ]
  }
]

In [29]:
with open('./data/library.json', 'r') as f:
    reloaded_library = json.load(f)

reloaded_library

[{'title': 'The Prophet',
  'author': 'Khalil Gibran',
  'genre': 'poetry',
  'tags': ['religion',
   'spirituality',
   'philosophy',
   'Lebanon',
   'Arabic',
   'Middle East'],
  'book_id': '811.19',
  'copies': [{'edition_year': 1996, 'checkouts': 486, 'borrowed': False},
   {'edition_year': 1996, 'checkouts': 443, 'borrowed': False}]},
 {'title': 'The Little Prince',
  'author': 'Antoine de Saint-Exupery',
  'genre': 'children',
  'tags': ['fantasy', 'France', 'philosophy', 'illustrated', 'fable'],
  'id': '843.912',
  'copies': [{'edition_year': 1983,
    'checkouts': 634,
    'borrowed': True,
    'due_date': '2017/02/02'},
   {'edition_year': 2015, 'checkouts': 41, 'borrowed': False}]}]

In [30]:
reloaded_library == library

True

In [31]:
# note that if we loaded it in without JSON
# the file would be interpreted as plain text

with open('./data/library.json', 'r') as f:
    library_string = f.read()

# this isn't what we want
library_string

'[\n  {\n    "title": "The Prophet",\n    "author": "Khalil Gibran",\n    "genre": "poetry",\n    "tags": [\n      "religion",\n      "spirituality",\n      "philosophy",\n      "Lebanon",\n      "Arabic",\n      "Middle East"\n    ],\n    "book_id": "811.19",\n    "copies": [\n      {\n        "edition_year": 1996,\n        "checkouts": 486,\n        "borrowed": false\n      },\n      {\n        "edition_year": 1996,\n        "checkouts": 443,\n        "borrowed": false\n      }\n    ]\n  },\n  {\n    "title": "The Little Prince",\n    "author": "Antoine de Saint-Exupery",\n    "genre": "children",\n    "tags": [\n      "fantasy",\n      "France",\n      "philosophy",\n      "illustrated",\n      "fable"\n    ],\n    "id": "843.912",\n    "copies": [\n      {\n        "edition_year": 1983,\n        "checkouts": 634,\n        "borrowed": true,\n        "due_date": "2017/02/02"\n      },\n      {\n        "edition_year": 2015,\n        "checkouts": 41,\n        "borrowed": false\n      

In [32]:
json.loads(library_string)

[{'title': 'The Prophet',
  'author': 'Khalil Gibran',
  'genre': 'poetry',
  'tags': ['religion',
   'spirituality',
   'philosophy',
   'Lebanon',
   'Arabic',
   'Middle East'],
  'book_id': '811.19',
  'copies': [{'edition_year': 1996, 'checkouts': 486, 'borrowed': False},
   {'edition_year': 1996, 'checkouts': 443, 'borrowed': False}]},
 {'title': 'The Little Prince',
  'author': 'Antoine de Saint-Exupery',
  'genre': 'children',
  'tags': ['fantasy', 'France', 'philosophy', 'illustrated', 'fable'],
  'id': '843.912',
  'copies': [{'edition_year': 1983,
    'checkouts': 634,
    'borrowed': True,
    'due_date': '2017/02/02'},
   {'edition_year': 2015, 'checkouts': 41, 'borrowed': False}]}]

In [33]:
# Pandas can also read_json
# notice how it constructs the table
# does it represent the data well?
import pandas as pd
pd.read_json('./data/library.json')

,title,author,genre,tags,book_id,copies,id
0,The Prophet,Khalil Gibran,poetry,"[religion, spirituality, philosophy, Lebanon, ...",811.19,"[{'edition_year': 1996, 'checkouts': 486, 'bor...",NaN
1,The Little Prince,Antoine de Saint-Exupery,children,"[fantasy, France, philosophy, illustrated, fable]",NaN,"[{'edition_year': 1983, 'checkouts': 634, 'bor...",843.912


In [34]:
# and to_json
df.to_json('./data/example_df.json')

!head ./data/example_df.json

{"name":{"0":"Dylan","1":"Terrence","2":"Mya"},"age":{"0":28,"1":54,"2":31}}

We can download JSON files many ways. Sometimes we will download it manually, but we can also use `wget` like we did for the CSV example. Often we'll connect to a website's API which will respond using JSON.

Panda's `read_json` method is capable of connecting directly to a URL (whether it's the address of a JSON file or an API connection) and reading the JSON without saving the file to our computer.

In [35]:
pd.read_json('https://api.github.com/repos/pydata/pandas/issues?per_page=5')

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,milestone,comments,created_at,updated_at,closed_at,author_association,active_lock_reason,pull_request,body,performed_via_github_app
0,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/42787,955297453,MDExOlB1bGxSZXF1ZXN0Njk5MDY4OTA5,42787,BUG: 1D slices over extension types turn into ...,...,None,0,2021-07-28 23:05:15+00:00,2021-07-28 23:05:43+00:00,NaT,CONTRIBUTOR,NaN,{'url': 'https://api.github.com/repos/pandas-d...,- [X] closes #42430\r\n- [X] tests added / pas...,NaN
1,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/42786,955290695,MDExOlB1bGxSZXF1ZXN0Njk5MDYzMDY4,42786,Backport PR #42762 on branch 1.3.x (REG: DataF...,...,{'url': 'https://api.github.com/repos/pandas-d...,0,2021-07-28 22:50:10+00:00,2021-07-28 22:50:11+00:00,NaT,NONE,NaN,{'url': 'https://api.github.com/repos/pandas-d...,Backport PR #42762: REG: DataFrame.agg where f...,NaN
2,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/42785,955210497,MDU6SXNzdWU5NTUyMTA0OTc=,42785,BUG: Multiindex Categorical type lost after di...,...,None,0,2021-07-28 20:29:52+00:00,2021-07-28 20:58:58+00:00,NaT,NONE,NaN,NaN,- [X] I have checked that this issue has not a...,NaN
3,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/42784,955189452,MDU6SXNzdWU5NTUxODk0NTI=,42784,ENH: Support Left Semi Joins,...,None,1,2021-07-28 19:59:06+00:00,2021-07-28 20:02:08+00:00,NaT,NONE,NaN,NaN,#### Is your feature request related to a prob...,NaN
4,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/42783,955170551,MDExOlB1bGxSZXF1ZXN0Njk4OTU5NzE5,42783,TST: Fix doctests in style.py,...,{'url': 'https://api.github.com/repos/pandas-d...,1,2021-07-28 19:29:59+00:00,2021-07-28 22:51:39+00:00,NaT,NONE,NaN,{'url': 'https://api.github.com/repos/pandas-d...,The following commits fix the doctests of the ...,NaN


## Compressed files (Gzip)

Another way we save storage and network resources is by using **compression**. Many times data sets will contain patterns that can be used to reduce the amount of space needed to store the information.

A simple example is the following list of numbers: 10, 10, 10, 2, 3, 3, 3, 3, 3, 50, 50, 1, 1, 50, 10, 10, 10, 10

Rather than writing out the full list of numbers (18 integers), we can represent the same information with only 14 numbers: (3, 10), (1, 2), (5, 3), (2, 50), (2, 1), (1, 50), (4, 10)

Here the first number in each pair is the number of repetitions, and the second number in the pair is the actual value. We've successfully reduced the amount of numbers we need to represent the same data. Most forms of compression use a similar idea, although actual implementations are usually more complex.

In the world of data science, the most common compression is Gzip (which uses the [deflate algorithm](http://www.infinitepartitions.com/art001.html)). Gzip files end with the extension `.gz`.

In [36]:
!wget -P ./data/ https://archive.org/stream/TheEpicofGilgamesh_201606/eog_djvu.txt

--2021-07-29 00:56:40--  https://archive.org/stream/TheEpicofGilgamesh_201606/eog_djvu.txt
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘./data/eog_djvu.txt.1’

eog_djvu.txt.1          [ <=>                ] 203.17K  1.15MB/s    in 0.2s    

2021-07-29 00:56:40 (1.15 MB/s) - ‘./data/eog_djvu.txt.1’ saved [208050]



In [37]:
import gzip

with open('./data/eog_djvu.txt', 'r') as f:
    text = f.read()

with gzip.open('./data/eog_djvu.txt.gz', 'wb') as f:
    f.write(text.encode('utf-8'))

!ls -lh ./data/eog*

-rw-rw-r-- 1 jovyan users 193K Jul 28 01:15 ./data/eog_djvu_scrambled.txt.gz
-rw-rw-r-- 1 jovyan users 204K Jul 28 01:08 ./data/eog_djvu.txt
-rw-r--r-- 1 jovyan users 204K Jul 29 00:56 ./data/eog_djvu.txt.1
-rw-rw-r-- 1 jovyan users  60K Jul 29 00:56 ./data/eog_djvu.txt.gz


We were able to compress the text of The Epic of Gilgamesh to a third of its original size! Remember that compression depends on patterns in the data. Language has a lot of patterns, but what would happen if we scrambled all the letters in the text?

In [38]:
import numpy as np

with gzip.open('./data/eog_djvu_scrambled.txt.gz', 'wb') as f:
    f.write(np.random.permutation(list(text)))

!ls -lh ./data/eog*

-rw-rw-r-- 1 jovyan users 192K Jul 29 00:56 ./data/eog_djvu_scrambled.txt.gz
-rw-rw-r-- 1 jovyan users 204K Jul 28 01:08 ./data/eog_djvu.txt
-rw-r--r-- 1 jovyan users 204K Jul 29 00:56 ./data/eog_djvu.txt.1
-rw-rw-r-- 1 jovyan users  60K Jul 29 00:56 ./data/eog_djvu.txt.gz


The scrambled version only compressed to two-thirds the size of the original. Compression won't perform very well on random data. Compression also doesn't work very well on data that is already small.

In [39]:
short_text = 'Hello'

with open('./data/short_text.txt', 'w') as f:
    f.write(short_text)

with gzip.open('./data/short_text.txt.gz', 'wb') as f:
    f.write(short_text.encode('utf-8'))

!ls -lh ./data/short_text*

-rw-rw-r-- 1 jovyan users  5 Jul 29 00:56 ./data/short_text.txt
-rw-rw-r-- 1 jovyan users 40 Jul 29 00:56 ./data/short_text.txt.gz


The compressed file is bigger than the plain text! That's because the compressed file includes a header, which takes up a small amount of extra space. Also, since the text is so short, it's not possible to use patterns to represent the text more efficiently. Therefore we usually save compression for large files.

You may have noticed that when we write Gzip files, we have been using a `'wb'` flag instead of a plain `'w'` flag. This is because Gzip is not plain text. When compressing the file we write _binary_ files. The files are not readable as plain text.

In [40]:
# we have to uncompress the file
# before we can read it

!cat ./data/short_text.txt.gz

���a�short_text.txt �H��� ����   

We should only use `'w'` for plain text files (which includes CSV and JSON). Using `'w'` instead of `'wb'` for Gzip files, or other files which are not plain text (e.g. images), could damage the file.

## Serialization (`pickle`)

Often we will want to save our work in Python and come back to it later. However, that work might be a machine learning model or some other complex object in Python. How do we save complex Python objects? Python has a module for this purpose called `pickle`. We can use `pickle` to write a binary file that contains all the information about a Python object. Later we can load that pickle file and reconstruct the object in Python.

In [41]:
pickle_example = ['hello', {'a': 23, 'b': True}, (1, 2, 3), [['dogs', 'cats'], None]]

In [42]:
%%expect_exception TypeError

# we can't save this as text
with open('./data/pickle_example.txt', 'w') as f:
    f.write(pickle_example)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-42-a9c72e388869> in <module>
      1 # we can't save this as text
      2 with open('./data/pickle_example.txt', 'w') as f:
----> 3     f.write(pickle_example)

TypeError: write() argument must be str, not list


In [53]:
with open('./data/pickle_example.json', 'w') as f:
    json.dump(pickle_example, f)

In [55]:
with open('./data/pickle_example.json', 'r') as f:
    pickle_json = json.load(f)

pickle_json

['hello', {'a': 23, 'b': True}, [1, 2, 3], [['dogs', 'cats'], None]]

In [57]:
import pickle

with open('./data/pickle_example.pkl', 'wb') as f:
    pickle.dump(pickle_example, f)
with open('./data/pickle_example.pkl', 'rb') as f:
    reloaded_pickle = pickle.load(f)

reloaded_pickle

['hello', {'a': 23, 'b': True}, (1, 2, 3), [['dogs', 'cats'], None]]

In [58]:
# the reloaded example is the same as the original

reloaded_example == pickle_example

True

Pickle is an important tool for data scientists. Data processing and training machine learning models can take a long time, and it is useful to save checkpoints.

Pandas also has `to_pickle` and `read_pickle` methods.

## NumPy file formats

NumPy also has methods for saving and loading data. They are straightforward to use. You may encounter these when working with certain machine learning libraries that require data be stored in NumPy arrays. NumPy arrays are also often used when working with image data.

In [45]:
sample_array = np.random.random((4, 4))
print(sample_array)

[[0.23295745 0.60009061 0.03963324 0.59982804]
 [0.01143172 0.76065107 0.73689047 0.80328372]
 [0.97495103 0.92592755 0.71370862 0.23445034]
 [0.93248108 0.96763913 0.04027293 0.66634547]]


In [46]:
# to save as plain text
np.savetxt('./data/sample_array.txt', sample_array)

In [47]:
!cat ./data/sample_array.txt

2.329574470831695665e-01 6.000906124068353664e-01 3.963324484876951104e-02 5.998280407619300814e-01
1.143171578508073161e-02 7.606510748245880471e-01 7.368904715970719943e-01 8.032837207925469514e-01
9.749510292722854965e-01 9.259275480629530763e-01 7.137086243708963273e-01 2.344503350402145081e-01
9.324810765667151946e-01 9.676391299312721017e-01 4.027292768008683410e-02 6.663454673913564719e-01


In [48]:
print(np.loadtxt('./data/sample_array.txt'))

[[0.23295745 0.60009061 0.03963324 0.59982804]
 [0.01143172 0.76065107 0.73689047 0.80328372]
 [0.97495103 0.92592755 0.71370862 0.23445034]
 [0.93248108 0.96763913 0.04027293 0.66634547]]


In [49]:
# to save as compressed binary
np.save('./data/sample_array.npy', sample_array)

In [50]:
!cat ./data/sample_array.npy

�NUMPY v {'descr': '<f8', 'fortran_order': False, 'shape': (4, 4), }                                                          
�zJ����?)�]:�3�?@n��J�?�zI��1�?�y�i�?L>A�@W�?EʇS���?h�����?ա=��2�?Ѫ,�2��?zx���?�X��w�?�������?�O,V���?���6���?(ɺ�R�?

In [51]:
print(np.load('./data/sample_array.npy'))

[[0.23295745 0.60009061 0.03963324 0.59982804]
 [0.01143172 0.76065107 0.73689047 0.80328372]
 [0.97495103 0.92592755 0.71370862 0.23445034]
 [0.93248108 0.96763913 0.04027293 0.66634547]]


In [59]:
!ls -l ./data/sample_array.*

-rw-r--r-- 1 jovyan users 256 Jul 29 00:56 ./data/sample_array.npy
-rw-r--r-- 1 jovyan users 400 Jul 29 00:56 ./data/sample_array.txt


## Topics used by not discussed:
- BASH commands (!)
- `wget`
- `str.split()`
- APIs

*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*